In [177]:
import pandas as pd
import pg8000
from selenium import webdriver
import numpy as np
import re
import time

In [178]:
#conn = pg8000.connect(database="movies", user="mathieurudaz")
#cursor = conn.cursor()

In [179]:
browser = webdriver.Chrome('/Users/mathieurudaz/Desktop/LEDE/chromedriver')

In [180]:
df_films = pd.read_csv('films-psql.csv', index_col='id')

df_directors_raw = pd.read_csv('directors_selenium_raw.csv')
df_films_raw = pd.read_csv('films_selenium_raw.csv')
df_genres_raw = pd.read_csv('genres_selenium_raw.csv')
df_directions_raw = pd.read_csv('directions_selenium_raw.csv')

In [181]:
print(len(df_directors_raw), len(df_films_raw), len(df_genres_raw), len(df_directions_raw))

442 577 1385 627


In [182]:
directors = [(str(d['name']) + str(float(d['year_of_birth'])) + str(d['nationality'])) for i, d in df_directors_raw.iterrows()]
genres = df_genres_raw['id'].tolist()
directions = df_directions_raw['id'].tolist()

In [183]:
directors

['David Dessites1973.0France',
 'Kar-Wai Wong1956.0China',
 'Terrence Malick1943.0USA',
 'Edward Yang1947.0China',
 'Jean-Luc Godard1930.0France',
 'Mohammad Rasoulof1973.0Iran',
 'Raoul Ruiz1941.0Chile',
 'Abbas Kiarostami1940.0Iran',
 'Johnnie To1955.0Hong Kong',
 'Carlos Saura1932.0Spain',
 'Michel Gondry1963.0France',
 'Apichatpong Weerasethakul1970.0Thailand',
 'Hayao Miyazaki1941.0Japan',
 'Joshua Oppenheimer1974.0USA',
 'Anonymousnannan',
 'Christine Cynnnannan',
 'Wes Anderson1969.0USA',
 'Zhangke Jia1970.0China',
 'Gus Van Sant1952.0USA',
 'Kathryn Bigelow1951.0USA',
 'David Cronenberg1943.0Canada',
 'Sarah Polley1979.0Canada',
 'Martin Campbell1943.0New Zealand',
 'Miguel Gomes1972.0Portugal',
 'Pablo Berger1963.0Spain',
 'Courtney Hunt1964.0USA',
 'Robert Altman1925.0USA',
 'Christopher Nolan1970.0UK',
 'Guillermo del Toro1964.0Mexico',
 'Michael Haneke1942.0Germany',
 'Werner Herzog1942.0Germany',
 'Cristian Mungiu1968.0Romania',
 'Leos Carax1960.0France',
 'Claude Lanzmann

In [184]:
# As it seems impossible to select text only with Selenium as BeautifulSoup does,
# this function extracts the text without the span tags.
def get_text_without_tags(web_element):
    text = web_element.text
    for t in web_element.find_elements_by_xpath('.//*'):
        text = text.replace(t.text, '')
    return text.strip()

def xpath_exists(xpath):
    try:
        browser.find_element_by_xpath(xpath)
        return browser.find_element_by_xpath(xpath)
    except:
        return False

def xpath_text_content(xpath):
    try:
        browser.find_element_by_xpath(xpath)
        return browser.find_element_by_xpath(xpath).text
    except:
        return np.nan    
      
    
for index, film in df_films[586:593].iterrows():
    
    # List of directors for this film
    this_film_directors = []
    # List of genres for this films
    this_film_genres = []
    
    # Print the current film index and the total of films to be processed
    print(index, '/', len(df_films))
        
    # Submit the film title in the search box
    browser.get('http://www.imdb.com')
    browser.find_element_by_xpath('//*[@id="navbar-query"]').send_keys(film['title'] + ' (' + str(film['release_year']) + ')');
    browser.find_element_by_xpath('//*[@id="navbar-form"]').submit()
    
    # If there is a result, let's assume it is the film we are looking for
    search_result = xpath_exists('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a')
    if search_result:
        time.sleep(1)
        search_result.click()
    # else save the film in the mismatches file and go to the next film
    else:
        mismatches_file = open('mismatches_selenium_raw.csv', 'a')
        mismatches_file.write(
            "\n" + str(film['title'] + ',' + 'NOT FOUND'))
        mismatches_file.close()
        continue
    
    # Get the film title
    found_title = get_text_without_tags(browser.find_element_by_xpath('//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/h1'))
    
    # Compare it to the BBC title to be sure it's the correct film, if not, save the title
    if film['title'] != found_title:
        mismatches_file = open('mismatches_selenium_raw.csv', 'a')
        mismatches_file.write(
            "\n" + str(film['title'] + ',' + found_title))
        mismatches_file.close()
        
    # Get the rating
    rating_element = xpath_exists('//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[1]/div[1]/div[1]/strong/span')
    if rating_element:
        rating = rating_element.text
        
    # Get the duration
    duration_element = xpath_exists('//*[@id="title-overview-widget"]/div[2]/div[2]/div/div[2]/div[2]/div/time')
    if duration_element:
        duration = duration_element.get_attribute('datetime')
    else:
        duration = np.nan
    
    # Get the genres
    genres_container = xpath_exists('//*[@id="titleStoryLine"]/div[@itemprop="genre"]')
    if genres_container:
        for genre_element in genres_container.find_elements_by_tag_name('a'):
            this_film_genres.append(genre_element.text)
    
    # if 'See full cast' link exists
    full_cast_element = xpath_exists('//*[@id="titleCast"]/div[@class="see-more"]/a')
    if full_cast_element:
        
        # Click on the link to reach the directors list
        full_cast_element.click()

        # Get all the directors links
        directors_link = browser.find_element_by_xpath('//*[@id="fullcredits_content"]/table[1]/tbody').find_elements_by_tag_name('a')
        directors_linktext = []

        # Store these links
        for director in directors_link:
            directors_linktext.append(director.text)

        # Click on each of them to get the director data
        for director_text in directors_linktext:
            time.sleep(2)
            browser.find_element_by_link_text(director_text).click()

            name = xpath_text_content('//*/h1/span')
            year_of_birth = xpath_text_content('//*[@id="name-born-info"]/time')
            nationality = xpath_text_content('//*[@id="name-born-info"]/a')

            # Add the director to the list
            this_film_directors.append({
                'name': name,
                'year_of_birth': year_of_birth,
                'nationality': nationality
            })

            # Return to the list of directors
            browser.back()            
            
    else:
        director_link = xpath_exists('//*[@itemprop="director"]/a')
        if director_link:
            director_link.click()
        
        name = xpath_text_content('//*/h1/span')
        year_of_birth = xpath_text_content('//*[@id="name-born-info"]/time')
        nationality = xpath_text_content('//*[@id="name-born-info"]/a')
        
        # Add the director to the list
        this_film_directors.append({
            'name': name,
            'year_of_birth': year_of_birth,
            'nationality': nationality
        })
    
    directors_file = open('directors_selenium_raw.csv', 'a')
    directions_file = open('directions_selenium_raw.csv', 'a')

    for director in this_film_directors:

        # Extract the year of birth
        year_of_birth = re.search('(\d\d\d\d)$', str(director['year_of_birth']))
        if year_of_birth:
            director['year_of_birth'] = year_of_birth.group(1)

        # Extract the nationality
        nationality = re.search('^.*,\s*(.*)$', str(director['nationality']))
        if nationality:
            director['nationality'] = nationality.group(1)            
        
        # Unique ID for the director
        imdb_scrap_director_id = (director['name'] + 
                                  str(float(director['year_of_birth'])) +
                                  str(director['nationality']))
                
        # Add the director to the database and the backup file if not already in it
        if imdb_scrap_director_id not in directors:
            
            # Write the row in the csv file
            directors_file.write(
                "\n" + str(len(directors)) + ',' +
                str(director['name']) + ',' + 
                str(director['year_of_birth']) + ',' + 
                str(director['nationality']))
            
            # Update the list of directors
            directors.append(imdb_scrap_director_id)

        # Add the relation of the director with this movie
        next_direction_id = (int(directions[len(directions)-1]) + 1 if len(directions) > 0 else '0')
        directions_file.write(
            "\n" + str(next_direction_id) + ',' +
            str(next(index for (index, d) in enumerate(directors) if d == imdb_scrap_director_id)) + ',' + 
            str(index))
        directions.append(next_direction_id)
        
    directors_file.close()
    directions_file.close()
    
    # Save the film
    films_file = open('films_selenium_raw.csv', 'a')
    films_file.write(
        "\n" + str(index) + ',' +
        '"' + str(film['title']) + '",' +
        str(film['release_year']) + ',' +
        str(rating) + ',' + 
        str(duration))
    films_file.close()
    
    # Save the genres
    genres_file = open('genres_selenium_raw.csv', 'a')    
    for genre in this_film_genres:
        next_genre_id = (int(genres[len(genres)-1]) + 1 if len(genres) > 0 else '0')
        genres_file.write(
            "\n" + str(next_genre_id) + ',' +
            str(index) + ',' +
            str(genre))
        genres.append(next_genre_id)
    genres_file.close()


588 / 593
589 / 593
590 / 593
591 / 593
592 / 593
593 / 593
594 / 593
